In [10]:
import sys, os

# force PYTHONPATH to look into the project directory for modules
rootdir = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, rootdir)
sys.path.insert(0, rootdir + '/genotypooler')

from genotypooler.poolSNPs import poolvcf
from genotypooler.poolSNPs.pooler import Design

# Pooling-imputation performance from real bead chip data

Performs pooling simulation and imputation on data from the chromosome 1 of samples in the NIAB Diverse MAGIC wheat population.
There 1170 markers on the chromosome and 496 samples of inbred lines in the study population STU.
The reference panel PNL consists in 8 founder individuals.
Imputation is done with Beagle 4.1 (`modelscale=1.5`, other parameters set to default value), as well as a genetic map.
For imputation with prophaser, we refer the user to the [`poolimputeSNPs`](https://github.com/camcl/poolimputeSNPs) repository.


In [2]:
print('Check number of samples')
%sx bcftools query -l Chr1.SNPs.pruned.nomiss.vcf.gz | wc -l

Check number of samples


['496']

In [3]:
print('Check number of markers')
%sx bcftools view -H Chr1.SNPs.pruned.nomiss.vcf.gz | wc -l

Check number of markers


['1170']

In [4]:
print('Apply pooling on the study population')
# get processor characteristics on Linux-based OS
%sx cat /proc/cpuinfo  | grep 'name'| uniq
# model name	: Intel(R) Core(TM) i7-7600U CPU @ 2.70GHz

Apply pooling on the study population


['model name\t: Intel(R) Core(TM) i7-10850H CPU @ 2.70GHz']

In [5]:
#pooling simulation
%sx python3 -u pooling-ex.py Chr1.SNPs.pruned.nomiss.vcf.gz Chr1.SNPs.pruned.nomiss.pooled.vcf.gz GP
# Time for pooling 1170 variants = ca. 18 sec

['',
 '*******************************************************************************',
 'Input file = Chr1.SNPs.pruned.nomiss.vcf.gz',
 'Output file = Chr1.SNPs.pruned.nomiss.pooled.vcf.gz',
 '*******************************************************************************',
 '',
 'Pooling data in /home/camille/MagicWheat/src/genotypooler/examples/Chr1.SNPs.pruned.nomiss.vcf.gz',
 '1 variants processed in 000.02 sec..............................................',
 '1001 variants processed in 015.78 sec...........................................',
 '',
 'Writing metadata in /home/camille/MagicWheat/src/genotypooler/examples/Chr1.SNPs.pruned.nomiss.pooled.vcf',
 '',
 'Writing data in /home/camille/MagicWheat/src/genotypooler/examples/Chr1.SNPs.pruned.nomiss.pooled.vcf',
 'Writing data in /home/camille/MagicWheat/src/genotypooler/examples/Chr1.SNPs.pruned.nomiss.pooled.vcf: Done',
 '/home/camille/MagicWheat/src/genotypooler/examples/Chr1.SNPs.pruned.nomiss.pooled.vcf.gz:',
 ' File create

In [6]:
print('Impute missing genotypes in the pooled file')
%sx bash beagle_magic_wheat_map.sh

Impute missing genotypes in the pooled file


['Contigs in the reference file',
 '.................................................................................',
 'Chromosome  1    Startpos = 1188258    Endpos = 593401002',
 '',
 '',
 'Check FORMAT field in files for imputation',
 '.................................................................................',
 'FORMAT in reference panel:  GT',
 'FORMAT in target:  GL',
 '',
 '',
 'Check number of samples and number of markers in files for imputation',
 '.................................................................................',
 'reference:',
 '16',
 '',
 'target:',
 '496',
 '',
 '',
 'Phase reference and target with BEAGLE',
 '.................................................................................',
 'Beagle .jar file used at: ../bin/beagle.11Mar19.69c.jar',
 '',
 'FORMAT in the phased ref file: GT',
 'beagle.11Mar19.69c.jar (version 4.1)',
 'Copyright (C) 2014-2015 Brian L. Browning',
 'Enter "java -jar beagle.11Mar19.69c.jar" for a summary of command 

In [7]:
# Verify files created at the different phasing and imputation steps
assert os.path.exists('Chr1.SNPs.pruned.nomiss.pooled.unphased.vcf.gz')
assert os.path.exists('Chr1.SNPs.pruned.nomiss.pooled.phased.vcf.gz')
assert os.path.exists('Chr1.SNPs.pruned.nomiss.pooled.dedup.vcf.gz')
assert os.path.exists('Chr1.SNPs.pruned.nomiss.pooled.imputed.vcf.gz')

AssertionError: 

### Compute results with customized metrics
Plot concordance and cross-entropy after imputation. Count number of markers correctly decoded and correctly imputed.

> **NB**: If wished, you may first edit the paths in `examples/argsfile_example.txt` to absolute paths that correspond to your local OS and settings.

In [21]:
%sx cd ../genotypooler/graphtools && python3 -u quantiles_plots.py @../../examples/argsfile_example.txt

['',
 '*******************************************************************************',
 'The following arguments were parsed from file:',
 '',
 "Namespace(bins=0.01, compute=1, date='../../examples/beagle-vs-prophaser', imp1='../../examples/beagle-with-map/Chr1.SNPs.pruned.nomiss.pooled.imputed.vcf.gz', imp2='../../examples/prophaser/Chr1.SNPs.pruned.nomiss.pooled.imputed.vcf.gz', pathout='../../examples/results', rollwin=5, truegl='../../examples/Chr1.SNPs.pruned.nomiss.gl.vcf.gz', truegt='../../examples/Chr1.SNPs.pruned.nomiss.vcf.gz')",
 '',
 '*******************************************************************************',
 '',
 'Data written to ../../examples/results/../../examples/beagle-vs-prophaser',
 '',
 '1170 variants from 496 samples read from Chr1.SNPs.pruned.nomiss.vcf.gz',
 '',
 '1170 variants from 496 samples read from Chr1.SNPs.pruned.nomiss.pooled.imputed.vcf.gz',
 '',
 '1170 variants from 496 samples read from Chr1.SNPs.pruned.nomiss.gl.vcf.gz',
 '',
 '1170 variant

In [13]:
%sx pwd

#TODO: tables, quantiles1_plot

["python3: can't open file 'quantiles_plots.py': [Errno 2] No such file or directory"]

In [37]:
# Verify files created at the different phasing and imputation steps
#assert os.path.exists('imputation_quality_gtgl.pdf')

